In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import plotly.graph_objects as go
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [2]:
summoner = 'Saikki kusuo'
key = 'RGAPI-9724b32a-f354-408c-8cde-8fdcc35e01fa'

## Extracting Functions

In [3]:
# Summoner v4, /lol/summoner/v4/summoners/by-name/{summonerName}
# Get summoner info by name using API key and summoner name

def get_summoner_info(summoner_name, apikey):
    # This function takes the summoner name and the API Key
    # and retrieves its account id
    url = f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    html = requests.get(url,
                       params={'api_key': apikey})
    json = html.json()
    df = pd.DataFrame(json, index=[0])
    
    return df

In [4]:
def get_matchlist(accountid, api):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    df = pd.DataFrame(json['matches'])
    
    return df

In [5]:
def get_match_info(matchid):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matches/{matchid}'
    html = requests.get(url,
                    params={'api_key': key})
    json = html.json()
    
    return json

In [6]:
def get_players_info(json):
        
    players = pd.DataFrame()
    
    for participant in range(10):
        df = pd.DataFrame.from_dict(json['participants'][participant]['stats'], orient='index').T
        df.insert(loc=0, column='championId', value= json['participants'][participant]['championId'])
        df.insert(loc=1, column='teamId', value=json['participants'][participant]['teamId'])
        
        dff = pd.DataFrame.from_dict(json['participantIdentities'][participant]['player'], orient='index').T
        dff.insert(loc=0, column='participantId', value= json['participantIdentities'][participant]['participantId'])
        df_final = pd.merge(df, dff, on='participantId')
        
        players = players.append(df_final)
    
    players.insert(loc=0, column='gameid', value=[json['gameId'] for value in range(len(players))])
        
        
        
    return players

In [7]:
def get_match_timeline(matchid):

    url= f'https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/{matchid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    
    return json

In [8]:
def participant_frames(json, playersinfo):
    ## Extracting frames from json

    all_frames = pd.DataFrame()

    for frame in list(range(len(json['frames']))):
        iterable = json['frames'][frame]['participantFrames']
        for elem in iterable:
            dic = {}
            for sub_elem in iterable[elem]:
                if type(iterable[elem][sub_elem]) == dict:
                    dic.update(iterable[elem][sub_elem])
                else:
                    dic[sub_elem] = iterable[elem][sub_elem]
            df = pd.DataFrame(dic, index=[0])
            df.insert(loc=0, column='timestamp', value=json['frames'][frame]['timestamp'])
            all_frames = all_frames.append(df, ignore_index=True)

    cols = ['timestamp', 'participantId', 'x', 'y', 'currentGold', 'totalGold', 'level',
            'xp', 'minionsKilled', 'jungleMinionsKilled',
            # 'dominionScore','teamScore',
            ]
    all_frames = all_frames[cols]
    
    teams = playersinfo[['participantId','teamId']]
    frames = pd.merge(all_frames, teams, on='participantId')
    frames['timestamp'] = (frames['timestamp'] / 60000).astype('int')

    return frames

In [9]:
def get_events(json):
    
    events = pd.DataFrame()
    
    frames = json['frames']
    for elem in range(len(frames)):
        events = events.append(pd.DataFrame(frames[elem]['events']))
    
    
    events['position_x'] = [elem.get('x') if type(elem) == dict else 'none' for elem in events['position']]
    events['position_y'] = [elem.get('y') if type(elem) == dict else 'none' for elem in events['position']]
    events.drop(columns='position', inplace=True)
    
    return events

In [10]:
summoner_info = get_summoner_info(summoner, key)
accountid = summoner_info['accountId'][0]
accountid

'djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c'

In [11]:
summoner_info = get_summoner_info(summoner, key)
summoner_info

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,9XoYu4fgD-ybcBawfY2XsMsGgvv9Av44llBjAhHFZk-nX5...,Saikki Kusuo,4249,1602858817000,193


In [12]:
match_list = get_matchlist(accountid, key)
match_list

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,EUW1,4879567107,222,420,13,1603210084969,DUO_CARRY,BOTTOM
1,EUW1,4879483641,202,420,13,1603205197335,DUO_CARRY,BOTTOM
2,EUW1,4879182781,157,420,13,1603190305010,DUO,MID
3,EUW1,4879210464,498,450,13,1603188959404,DUO,NONE
4,EUW1,4877596885,202,420,13,1603120271250,DUO_CARRY,BOTTOM
...,...,...,...,...,...,...,...,...
95,EUW1,4837079410,63,450,13,1601135321175,DUO_SUPPORT,TOP
96,EUW1,4837048701,98,420,13,1601132231280,NONE,JUNGLE
97,EUW1,4836878714,238,420,13,1601128794876,DUO,TOP
98,EUW1,4836645733,64,420,13,1601124381254,NONE,JUNGLE


In [13]:
list(match_list['gameId'].head(10))

[4879567107,
 4879483641,
 4879182781,
 4879210464,
 4877596885,
 4877250033,
 4876228611,
 4876149459,
 4876010396,
 4875263982]

In [14]:
match_info = get_match_info(4836878714)

In [15]:
players_info = get_players_info(match_info)
players_info

,gameid,championId,teamId,participantId,win,item0,item1,item2,item3,item4,...,statPerk1,statPerk2,platformId,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,4836878714,75,100,1,False,3065,3078,3075,3111,3053,...,5002,5001,EUW1,bvnCbvyVbhflb4myUIHiY9S5loCFtGukU6JWwhJl82w1J68,BlackChaos2,9QVsWAnLfAEePJm9wS8PGv-21y0ka3IsNHq97w2jnhejFBJf,EUW1,bvnCbvyVbhflb4myUIHiY9S5loCFtGukU6JWwhJl82w1J68,/v1/stats/player_history/EUW1/235487147,3186
0,4836878714,104,100,2,False,1412,3046,3072,3006,3026,...,5008,5002,EUW1,EYLwmPGfd4E2POEi1C4BQugPcpdXu6cWObmNCQJ6k6STt_s,Eleanc,MaonGnpsB5B0IH6VaEIQQWa7VlHPahnwBda7FXdVWexVhoc,EUW1,EYLwmPGfd4E2POEi1C4BQugPcpdXu6cWObmNCQJ6k6STt_s,/v1/stats/player_history/EUW1/222153008,774
0,4836878714,99,100,3,False,3853,3285,3020,3165,3089,...,5008,5002,EUW1,IzXEB9k0kG3HsI39ryF1cLFF8Rg9tI3r3AMyfglVZ0fqrwE,angrypig331,b0smXotEGyqYSM3FjxLWQDeA4nO3oCB_tizvLCE2tYD9FqA,EUW1,IzXEB9k0kG3HsI39ryF1cLFF8Rg9tI3r3AMyfglVZ0fqrwE,/v1/stats/player_history/EUW1/214652021,1645
0,4836878714,238,100,4,False,3181,3142,3158,3033,3071,...,5008,5003,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,Saikki Kusuo,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,/v1/stats/player_history/EUW1/229546365,4249
0,4836878714,202,100,5,False,3026,3095,3031,3094,3009,...,5008,5002,EUW1,0RI3D0-sFaXlVcSnryMN2R6aheBouFi8YjujwGCLD-wpNA,John Chirac,jbI3HDjwPlktxAxLpyGyhBFk7BLC5Q8N0TnyNJplqE5eHEo,EUW1,0RI3D0-sFaXlVcSnryMN2R6aheBouFi8YjujwGCLD-wpNA,/v1/stats/player_history/EUW1/31288060,20
0,4836878714,54,200,6,True,3047,3075,1413,3110,3001,...,5008,5002,EUW1,-nIJJjBJOnb5F8vl8dyclBHMoL9vYzKeJPcBuLciQrM6A7M,An Italian JGL,NX62yZtC86KMcOX21Wx93-7w6eMcNFqQqnAOwk3muzo9XcQ,EUW1,-nIJJjBJOnb5F8vl8dyclBHMoL9vYzKeJPcBuLciQrM6A7M,/v1/stats/player_history/EUW1/224362586,4668
0,4836878714,517,200,7,True,3152,3285,3135,3020,3089,...,5008,5002,EUW1,c7noCl7HIhxIFRZljAxkEwHkWxAn7Z57RP0pUS1JSXtkt3E,RollingSomeGreen,RCfzOztx0NAkEADtWmZvDQFjmOKouvrIjL41akZ7gcyaZUY,EUW1,c7noCl7HIhxIFRZljAxkEwHkWxAn7Z57RP0pUS1JSXtkt3E,/v1/stats/player_history/EUW1/226271800,4495
0,4836878714,497,200,8,True,3860,2065,3107,3109,3117,...,5008,5002,EUW1,QPAUZH5jBj-WXjU6vTAynvrrF2MqG_zpVfzbB-lTdNFn2wc,Odolgan,Pp7Ng88AIs7aOwIYWeGVNJBA9mRTKiAaVQ0Vx4vrhWDkyjg,EUW1,QPAUZH5jBj-WXjU6vTAynvrrF2MqG_zpVfzbB-lTdNFn2wc,/v1/stats/player_history/EUW1/220794298,4630
0,4836878714,157,200,9,True,3026,3006,3046,3031,3033,...,5008,5003,EUW1,2LTE8gHjEq2Am1Xya38_JU-TV4otYd1Q7xYmyuq67kYii4c,NòMèrcy,Zmhe8jII-zXzu0d_6Q26VhGTg1FGlappR9kg4bONih-vhxk,EUW1,2LTE8gHjEq2Am1Xya38_JU-TV4otYd1Q7xYmyuq67kYii4c,/v1/stats/player_history/EUW1/220603622,2077
0,4836878714,875,200,10,True,1054,3077,3047,3071,3053,...,5008,5002,EUW1,9urZuBF9sr8q5elGQ4YrWsIx-cB1FCNU7LbHMcyHzxhdUgs,StatalEZ,n5IuRFfuMB-qMjNYaDLZumDh299sxYz0Ay39hhTgijYNtL4,EUW1,9urZuBF9sr8q5elGQ4YrWsIx-cB1FCNU7LbHMcyHzxhdUgs,/v1/stats/player_history/EUW1/206210487,4568


In [16]:
#list(players_info)

In [17]:
timeline = get_match_timeline(4836878714)

In [18]:
frames = participant_frames(timeline, players_info)
frames

,timestamp,participantId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,teamId
0,0,1,560.0,581.0,500,500,1,0,0,0,100
1,1,1,3611.0,9365.0,0,500,1,0,0,0,100
2,2,1,1798.0,11949.0,42,542,1,60,1,0,100
3,3,1,1682.0,10310.0,311,811,3,722,9,0,100
4,4,1,1295.0,10807.0,560,1060,4,1176,16,0,100
...,...,...,...,...,...,...,...,...,...,...,...
475,43,8,4334.0,3421.0,15,11897,18,20895,50,4,200
476,44,8,12311.0,13079.0,155,12037,18,20895,50,4,200
477,45,8,7506.0,8734.0,541,12424,18,22108,51,4,200
478,46,8,5723.0,5680.0,843,12725,18,23262,53,4,200


In [19]:
frames['timestamp'].max()

47

In [20]:
events = get_events(timeline)
events

,type,timestamp,participantId,itemId,skillSlot,levelUpType,wardType,creatorId,killerId,victimId,...,monsterType,monsterSubType,afterId,beforeId,teamId,buildingType,laneType,towerType,position_x,position_y
0,ITEM_PURCHASED,6651,5.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
1,ITEM_PURCHASED,6850,5.0,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
2,ITEM_PURCHASED,6982,1.0,1054.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
3,ITEM_PURCHASED,7147,1.0,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
4,ITEM_PURCHASED,7214,5.0,3340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,WARD_PLACED,2780997,NaN,NaN,NaN,NaN,UNDEFINED,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
6,CHAMPION_KILL,2782681,NaN,NaN,NaN,NaN,NaN,NaN,9.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4667,2674
7,CHAMPION_KILL,2785722,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4603,2329
8,CHAMPION_KILL,2789453,NaN,NaN,NaN,NaN,NaN,NaN,7.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2829,2372


In [21]:
data = frames.groupby(['timestamp', 'teamId']).sum().reset_index()[['timestamp', 'teamId', 'totalGold']]
data

,timestamp,teamId,totalGold
0,0,100,2500
1,0,200,2500
2,1,100,3035
3,1,200,2500
4,2,100,3850
...,...,...,...
91,45,200,87959
92,46,100,87362
93,46,200,89824
94,47,100,87983


In [22]:
team100gold = data[data['teamId'] == 100]
team100gold.rename(columns={"totalGold": "team100gold"}, inplace=True)
team200gold = data[data['teamId'] == 200][['timestamp', 'teamId', 'totalGold']]
team200gold.rename(columns={"totalGold": "team200gold"}, inplace=True)

/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
golddiff = pd.merge(team100gold, team200gold, on='timestamp')
golddiff['golddiff'] = golddiff['team100gold'] - golddiff['team200gold']
golddiff

,timestamp,teamId_x,team100gold,teamId_y,team200gold,golddiff
0,0,100,2500,200,2500,0
1,1,100,3035,200,2500,535
2,2,100,3850,200,2824,1026
3,3,100,5231,200,4091,1140
4,4,100,6443,200,5448,995
5,5,100,7672,200,6965,707
6,6,100,8905,200,8179,726
7,7,100,10140,200,10163,-23
8,8,100,12149,200,11224,925
9,9,100,13532,200,12607,925


In [24]:
def gold_diff(frames):
    data = frames.groupby(['timestamp', 'teamId']).sum().reset_index()[['timestamp', 'teamId', 'totalGold']]
    
    team100gold = data[data['teamId'] == 100]
    team100gold.rename(columns={"totalGold": "team100gold"}, inplace=True)
    team200gold = data[data['teamId'] == 200][['timestamp', 'teamId', 'totalGold']]
    team200gold.rename(columns={"totalGold": "team200gold"}, inplace=True)
    
    golddiff = pd.merge(team100gold, team200gold, on='timestamp')
    golddiff['golddiff'] = golddiff['team100gold'] - golddiff['team200gold']
    golddiff['team100golddiff'] = [a if a > 0 else 0 for a in golddiff['golddiff']]
    golddiff['team200golddiff'] = [a if a < 0 else 0 for a in golddiff['golddiff']]
    
    return golddiff

In [25]:
golddiff = gold_diff(frames)

/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
golddiff

,timestamp,teamId_x,team100gold,teamId_y,team200gold,golddiff,team100golddiff,team200golddiff
0,0,100,2500,200,2500,0,0,0
1,1,100,3035,200,2500,535,535,0
2,2,100,3850,200,2824,1026,1026,0
3,3,100,5231,200,4091,1140,1140,0
4,4,100,6443,200,5448,995,995,0
5,5,100,7672,200,6965,707,707,0
6,6,100,8905,200,8179,726,726,0
7,7,100,10140,200,10163,-23,0,-23
8,8,100,12149,200,11224,925,925,0
9,9,100,13532,200,12607,925,925,0


In [61]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=golddiff['timestamp'],
                         y=golddiff['team100golddiff'],
                         fill='tozeroy',
                         line_color='#229954'
                         )
                        )

fig.add_trace(go.Scatter(x=golddiff['timestamp'],
                         y=golddiff['team200golddiff'],
                         fill='tozeroy',
                         line_color='#E74C3C'
                         
                         )
                        )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=golddiff['timestamp'],
                         y=golddiff['golddiff'],
                         fill='tozeroy',
                         #mode='markers',
                         line_color=np.where(np.logical_and(golddiff['golddiff']>0), 'green', 'red')
                         )
                        )

TypeError: ufunc() missing 1 of 2required positional argument(s)